In [73]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import datetime

In [98]:
curr_dir = !pwd
domains_th = "10"
input_path = curr_dir[0]+"/../5.HMM_alter_align/domains_states_dicts/pfam-v30/"

#Read the list of domains
with open(curr_dir[0]+"/../5.domains_stats/filtered"+domains_th+"_list.pik", 'rb') as handle:
    filtered_domains_list = pickle.load(handle)
filtered_domains_list.sort()

#Read the canonic proteins table


In [104]:
mismatch_dict = defaultdict(list)
mismatch_count = 0
for domain_name in filtered_domains_list:
    
    dirfiles = !ls -t $input_path$domain_name
    filename = dirfiles[0]
    with open(input_path+domain_name+"/"+filename, 'rb') as handle:
        states_dict = pickle.load(handle)
    
    with open(curr_dir[0]+"/../4.parse_Uniprot/domains_canonic_prot/pfam-v30/"+domain_name+"_canonic_prot.pik", 'rb') as handle:
        canonic_protein = pickle.load(handle)
    
    for state in states_dict.keys():
        
        for d in states_dict[state]:
            
            #Find the protein ID
            gene = d["ens_gene"]
            protein = canonic_protein[gene]
            
            #Find JSD scores file by chromosome/gene id/protein id
            jsd_table = pd.read_csv(curr_dir[0]+"/../conservation_scores/proteins_ucsc_con/Homo_sapiens.GRCh37/"+d["chrom"]+"/"+gene+"/"+protein+".jsd.txt", skiprows=1, header=None, sep='\t')
            jsd_table.columns = ["Prot_idx", "AA", "JSD"]
            
            #Find the JSD of the corresponding protein position
            prot_pos = d["prot_pos"]
            prot_pos_adj = (prot_pos - 1) #Adjust the position because JSDs indices starts at 0.
            jsd_of_pos = float(jsd_table[jsd_table["Prot_idx"] == prot_pos_adj]["JSD"])
            
            #Sanity check: the AA match
            jsd_table_aa = jsd_table[jsd_table["Prot_idx"] == prot_pos_adj]["AA"].values[0]
            states_dict_aa = d["aa_ref"]
            if (jsd_table_aa.upper() != states_dict_aa.upper()):
                mismatch_count += 1
                mismatch_dict["chrom"].append(d["chrom"])
                mismatch_dict["gene"].append(gene)
                mismatch_dict["protein"].append(protein)
                mismatch_dict["prot_pos"].append(prot_pos)
                mismatch_dict["aa_ref"].append(states_dict_aa.upper())
                mismatch_dict["aa_jsd"].append(jsd_table_aa.upper())
                mismatch_dict["domain"].append(domain_name)
        
            d["100-way-BLOSUM_JSD"] = jsd_of_pos
    
    #Saving the updated dictionary
    !mkdir -p ext_features_dicts/pfam-v30/$domain_name
    
    with open(curr_dir[0]+"/ext_features_dicts/pfam-v30/"+domain_name+"/"+domain_name+"_hmm_states_dict_"+datetime.date.today().strftime("%m.%d.%y")+".pik", 'wb') as handle:
        pickle.dump(states_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    print "Finished "+domain_name

Finished 2OG-FeII_Oxy_3
Finished 7TM_GPCR_Srsx
Finished 7tm_1
Finished 7tm_2
Finished 7tm_3
Finished 7tm_4
Finished A2M
Finished A2M_N
Finished A2M_N_2
Finished A2M_comp
Finished A2M_recep
Finished AAA
Finished AAA_11
Finished AAA_12
Finished AAA_17
Finished AAA_18
Finished AAA_33
Finished AAA_5
Finished AAA_6
Finished AAA_7
Finished AAA_8
Finished AAA_9
Finished AA_permease
Finished AA_permease_2
Finished ABC2_membrane_3
Finished ABC_membrane
Finished ABC_tran
Finished ADAM_CR
Finished ADAM_spacer1
Finished ADH_N
Finished ADH_zinc_N
Finished ADK
Finished AMP-binding
Finished AMP-binding_C
Finished ANAPC3
Finished ANAPC4_WD40
Finished ANATO
Finished ANF_receptor
Finished APC_r
Finished APOBEC_C
Finished APOBEC_N
Finished ARID
Finished Aa_trans
Finished Abhydrolase_1
Finished Abhydrolase_6
Finished Acetyltransf_1
Finished Actin
Finished Acyl-CoA_dh_1
Finished Acyl-CoA_dh_2
Finished Acyl-CoA_dh_M
Finished Acyl-CoA_dh_N
Finished Acyltransferase
Finished Adap_comp_sub
Finished Adaptin_N
Fi

In [105]:
mismatch_df = pd.DataFrame.from_dict(mismatch_dict)
mismatch_df.to_csv(curr_dir[0]+"/aa_mismatch.csv", sep='\t')

In [95]:
jsd_table

,Prot_idx,AA,JSD
0,0,M,0.731142
1,1,K,0.669251
2,2,V,0.645092
3,3,D,0.676030
4,4,R,0.679480
5,5,T,0.615633
6,6,K,0.669251
7,7,L,0.606990
8,8,K,0.669251
9,9,K,0.669251
